# Lab 2

In [1]:
import os
import sys
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()
import pyspark

In [2]:
number_cores = 6
memory_gb = 16
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [3]:
!ls /users/trush/CSC496/PageRank/Lab2

data  Lab2.ipynb


In [33]:
auth_log = sc.textFile("/users/trush/CSC496/PageRank/Lab2/data")
auth_log.take(5)

['Nov 15 00:07:01 submitty CRON[31671]: pam_unix(cron:session): session opened for user submitty_daemon by (uid=0)',
 'Nov 15 00:07:01 submitty CRON[31671]: pam_unix(cron:session): session closed for user submitty_daemon',
 'Nov 15 00:07:21 submitty sshd[31682]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=218.92.0.248  user=root',
 'Nov 15 00:07:23 submitty sshd[31682]: Failed password for root from 218.92.0.248 port 11673 ssh2',
 'Nov 15 00:07:27 submitty sshd[31682]: Failed password for root from 218.92.0.248 port 11673 ssh2']

## 1. How many failed attempts to access the server as root are there? List all countries from which these attempts where carried out.

In [34]:
# Total failed attempts to login
#total_failed_attempts = auth_log.filter(lambda x: "sshd" in x)
#total_failed_attempts.count()

# Number of failed attempts to login as root
failed_attempts = auth_log.filter(lambda x: "root" in x and "sshd" in x)
failed_attempts.take(5)
# find distinct attempts by unique ip
#failed_attempts.count()

['Nov 15 00:07:21 submitty sshd[31682]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=218.92.0.248  user=root',
 'Nov 15 00:07:23 submitty sshd[31682]: Failed password for root from 218.92.0.248 port 11673 ssh2',
 'Nov 15 00:07:27 submitty sshd[31682]: Failed password for root from 218.92.0.248 port 11673 ssh2',
 'Nov 15 00:11:45 submitty sshd[31831]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=112.85.42.98  user=root',
 'Nov 15 00:11:47 submitty sshd[31831]: Failed password for root from 112.85.42.98 port 11028 ssh2']

In [35]:
# Countries where the failed attempts were carried out
# Split each literal to isolate the ip
# use python library to match ip to country
import re

countries_ip = failed_attempts.map(lambda line: re.split(pattern=r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})', string=line)[1])
countries_ip.take(5)

#countries_names = countries_ip.map(lambda line: get_country_name(line))
#countries_names.take(10)

['218.92.0.248',
 '218.92.0.248',
 '218.92.0.248',
 '112.85.42.98',
 '112.85.42.98']

## 2. How many failed attempts to access the server as non-root users are there? What are the attempted usernames? List all countries from which these attempts where carried out. 

In [31]:
# Number of failed attempts to login as non-root
failed_attempts_nonroot = auth_log.filter(lambda x: "root" not in x and "sshd" in x and "user unknown" not in x)
failed_attempts_nonroot_ip = failed_attempts_nonroot.map(lambda line: re.split(pattern=r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})', string=line)[1])
#failed_attempts.take(5)
#failed_attempts_nonroot.count()
#failed_attempts_nonroot_ip.take(5)
#failed_attempts_nonroot_usernames = failed_attempts_nonroot.map(lambda line: re.split(pattern=r'(user )', string=line).rsplit(' from'))
#failed_attempts_nonroot_usernames.take(5)

## 3. Through out the durations of the log files, which date has the highest number of attack attempts? Anything interesting about that particular date?

In [102]:
# Something to do with Chinese Labor Day (May 1)
dates = auth_log.map(lambda line: (re.split(pattern = r'\S{3}\W{1,2}\D{2}', string=line)[0], 1))
dates.take(5)
countsByKey = dates.countByKey()
#countsByKey